In [1]:
# Dependencies
from src.dataset import WarAndPeace
from src.encoder import WordToVector
from src.network import GutenNet
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
import torch
import numpy as np

In [2]:
# Path to manually cleaned dataset (War and Peace by Tolstoj)
WAR_AND_PEACE_PATH = './data/clean/war_and_peace_tolstoj.txt'

In [3]:
# Path to GloVe embedding
GLOVE_50_EMBEDDING = './data/embedding/glove.6B.50d.txt'
GLOVE_100_EMBEDDING = './data/embedding/glove.6B.100d.txt'
GLOVE_300_EMBEDDING = './data/embedding/glove.6B.300d.txt'

In [4]:
# Set cude as device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [5]:
# Initialize new dataset from file
ds = WarAndPeace(
    in_path=WAR_AND_PEACE_PATH,  # Path to text file
    crop_len=5 # Words window dimension
)

In [6]:
# Set encoder
ds.encoder = WordToVector.from_csv(
    in_path=GLOVE_50_EMBEDDING,  # Path to embeddings file
    dim=50,  # Dimension of the encoded word vectors
    words=ds.words  # The words which must be there
)

# Get vocabulary size and embedding dimensions
vocab_size, embedding_dim = ds.encoder.shape

In [7]:
# Create new network
net = GutenNet(
    vocab_size=vocab_size,  # Set vocabulary size
    embedding_dim=embedding_dim,  # Set embedding dimensions
    hidden_size=128,  # Set number of units per hidden layer
    num_layers=3,  # Set number of hidden layers
    dropout=0.4  # Set dropout
)

# Initialize embeddings
emb = np.array(ds.encoder.vectors)  # Get embedding vectors as numpy array
emb = emb / np.linalg.norm(emb)  # Normalize vectors
emb = torch.tensor(emb).float()  # Store vectors in a tensor
# Set embeddings pretrained parameters
net.emb.weight = nn.Parameter(emb)  # Set pretrained weigths
net.emb.weight.requires_grad = True  # Set weights update

# Put network on correct device
net.to(device)

# Train network on a dataset batch
def train_batch(net, batch_numbers, loss_fn, optimizer):
       
    # Get the labels (the last word of each sentence)
    labels_numbers = batch_numbers[:, -1]
    labels_encoded = net.emb.weight[labels_numbers]
    
    # Remove the labels from the input tensor
    net_input = batch_numbers[:, :-1]

    # Eventually clear previously recorded gradients
    optimizer.zero_grad()
    # Make forward pass (over batch)
    net_output, _ = net(net_input)

    # Evaluate loss only for last output
    loss = loss_fn(net_output[:, -1, :], labels_encoded)
    # Backward pass
    loss.backward()
    # Update
    optimizer.step()
    
    # Return average batch loss
    return float(loss.data)

In [8]:
# Define optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=0.001)

# Define loss function
loss_fn = nn.MSELoss()

# Define number of epochs
num_epochs = 1000
# Define batch size
batch_size = 5000

# Create a dataloader by using War and Peace dataset
dl = DataLoader(ds, batch_size=batch_size, shuffle=True)
# Loop through each batch for the given number of epochs
for epoch in range(num_epochs):
    # Loop through each batch in epoch
    for batch_numbers in dl:
        # Put batch to GPU
        batch_numbers = batch_numbers.to(device)
        # Retrieve loss for current batch
        loss = train_batch(net, batch_numbers, loss_fn, optimizer)
        # Remove batch from GPU
        del batch_numbers
    # Print out loss et each epoch
    print('Epoch nr. {:03d}'.format(epoch+1))
    print('Loss: {:f}'.format(loss))
    print()

Epoch nr. 001
Loss: 0.000128

Epoch nr. 002
Loss: 0.000030

Epoch nr. 003
Loss: 0.000015

Epoch nr. 004
Loss: 0.000015

Epoch nr. 005
Loss: 0.000004

Epoch nr. 006
Loss: 0.000003

Epoch nr. 007
Loss: 0.000002

Epoch nr. 008
Loss: 0.000001

Epoch nr. 009
Loss: 0.000001

Epoch nr. 010
Loss: 0.000000

Epoch nr. 011
Loss: 0.000000

Epoch nr. 012
Loss: 0.000000

Epoch nr. 013
Loss: 0.000000

Epoch nr. 014
Loss: 0.000000

Epoch nr. 015
Loss: 0.000000

Epoch nr. 016
Loss: 0.000000

Epoch nr. 017
Loss: 0.000000

Epoch nr. 018
Loss: 0.000000

Epoch nr. 019
Loss: 0.000000

Epoch nr. 020
Loss: 0.000000

Epoch nr. 021
Loss: 0.000000

Epoch nr. 022
Loss: 0.000000

Epoch nr. 023
Loss: 0.000000

Epoch nr. 024
Loss: 0.000000

Epoch nr. 025
Loss: 0.000000

Epoch nr. 026
Loss: 0.000000

Epoch nr. 027
Loss: 0.000000

Epoch nr. 028
Loss: 0.000000

Epoch nr. 029
Loss: 0.000000

Epoch nr. 030
Loss: 0.000000

Epoch nr. 031
Loss: 0.000000

Epoch nr. 032
Loss: 0.000000

Epoch nr. 033
Loss: 0.000000

Epoch nr. 

Epoch nr. 275
Loss: 0.000000

Epoch nr. 276
Loss: 0.000000

Epoch nr. 277
Loss: 0.000000

Epoch nr. 278
Loss: 0.000000

Epoch nr. 279
Loss: 0.000000

Epoch nr. 280
Loss: 0.000000

Epoch nr. 281
Loss: 0.000000

Epoch nr. 282
Loss: 0.000000

Epoch nr. 283
Loss: 0.000000

Epoch nr. 284
Loss: 0.000000

Epoch nr. 285
Loss: 0.000000

Epoch nr. 286
Loss: 0.000000

Epoch nr. 287
Loss: 0.000000

Epoch nr. 288
Loss: 0.000000

Epoch nr. 289
Loss: 0.000000

Epoch nr. 290
Loss: 0.000000

Epoch nr. 291
Loss: 0.000000

Epoch nr. 292
Loss: 0.000000

Epoch nr. 293
Loss: 0.000000

Epoch nr. 294
Loss: 0.000000

Epoch nr. 295
Loss: 0.000000

Epoch nr. 296
Loss: 0.000000

Epoch nr. 297
Loss: 0.000000

Epoch nr. 298
Loss: 0.000000

Epoch nr. 299
Loss: 0.000000

Epoch nr. 300
Loss: 0.000000

Epoch nr. 301
Loss: 0.000000

Epoch nr. 302
Loss: 0.000000

Epoch nr. 303
Loss: 0.000000

Epoch nr. 304
Loss: 0.000000

Epoch nr. 305
Loss: 0.000000

Epoch nr. 306
Loss: 0.000000

Epoch nr. 307
Loss: 0.000000

Epoch nr. 

Epoch nr. 549
Loss: 0.000000

Epoch nr. 550
Loss: 0.000000

Epoch nr. 551
Loss: 0.000000

Epoch nr. 552
Loss: 0.000000

Epoch nr. 553
Loss: 0.000000

Epoch nr. 554
Loss: 0.000000

Epoch nr. 555
Loss: 0.000000

Epoch nr. 556
Loss: 0.000000

Epoch nr. 557
Loss: 0.000000

Epoch nr. 558
Loss: 0.000000

Epoch nr. 559
Loss: 0.000000

Epoch nr. 560
Loss: 0.000000

Epoch nr. 561
Loss: 0.000000

Epoch nr. 562
Loss: 0.000000

Epoch nr. 563
Loss: 0.000000

Epoch nr. 564
Loss: 0.000000

Epoch nr. 565
Loss: 0.000000

Epoch nr. 566
Loss: 0.000000

Epoch nr. 567
Loss: 0.000000

Epoch nr. 568
Loss: 0.000000

Epoch nr. 569
Loss: 0.000000

Epoch nr. 570
Loss: 0.000000

Epoch nr. 571
Loss: 0.000000

Epoch nr. 572
Loss: 0.000000

Epoch nr. 573
Loss: 0.000000

Epoch nr. 574
Loss: 0.000000

Epoch nr. 575
Loss: 0.000000

Epoch nr. 576
Loss: 0.000000

Epoch nr. 577
Loss: 0.000000

Epoch nr. 578
Loss: 0.000000

Epoch nr. 579
Loss: 0.000000

Epoch nr. 580
Loss: 0.000000

Epoch nr. 581
Loss: 0.000000

Epoch nr. 

Epoch nr. 823
Loss: 0.000000

Epoch nr. 824
Loss: 0.000000

Epoch nr. 825
Loss: 0.000000

Epoch nr. 826
Loss: 0.000000

Epoch nr. 827
Loss: 0.000000

Epoch nr. 828
Loss: 0.000000

Epoch nr. 829
Loss: 0.000000

Epoch nr. 830
Loss: 0.000000

Epoch nr. 831
Loss: 0.000000

Epoch nr. 832
Loss: 0.000000

Epoch nr. 833
Loss: 0.000000

Epoch nr. 834
Loss: 0.000000

Epoch nr. 835
Loss: 0.000000

Epoch nr. 836
Loss: 0.000000

Epoch nr. 837
Loss: 0.000000

Epoch nr. 838
Loss: 0.000000

Epoch nr. 839
Loss: 0.000000

Epoch nr. 840
Loss: 0.000000

Epoch nr. 841
Loss: 0.000000

Epoch nr. 842
Loss: 0.000000

Epoch nr. 843
Loss: 0.000000

Epoch nr. 844
Loss: 0.000000

Epoch nr. 845
Loss: 0.000000

Epoch nr. 846
Loss: 0.000000

Epoch nr. 847
Loss: 0.000000

Epoch nr. 848
Loss: 0.000000

Epoch nr. 849
Loss: 0.000000

Epoch nr. 850
Loss: 0.000000

Epoch nr. 851
Loss: 0.000000

Epoch nr. 852
Loss: 0.000000

Epoch nr. 853
Loss: 0.000000

Epoch nr. 854
Loss: 0.000000

Epoch nr. 855
Loss: 0.000000

Epoch nr. 

In [9]:
# Test trained network

# Set seed text
seed_text = 'I don\'t know'
# Define length of generated text
words_len = 100

# Retrieve cpu device
cpu = torch.device('cpu')

# Move network back to cpu
net.to(cpu)
# Set network to evaluation mode
net.eval()
# Disable gradient computation
with torch.no_grad():
    
    # Update encoder vectors
    emb = net.emb.weight.detach().numpy()
    ds.encoder.vectors_mean = np.mean(emb, axis=0)  # Compute column mean
    ds.encoder.vectors_std = np.std(emb, axis=0)  # Compute column std dev
    ds.encoder.vectors = emb.tolist()
    
    # Clean text
    seed_text = WarAndPeace.clean_text(seed_text)
    # Split seed sentence into words
    seed_sentence = WarAndPeace.split_words([seed_text])
    # Initialize encoded sentence with random words
    seed_encoded = [[ds.encoder.encode('') for i in range(ds.crop_len-1)]]
    # Put actual words into encoded sequence entry
    for i in range(1, min(len(seed_encoded), ds.crop_len)):
        # Get - i-th word
        word_decoded = seed_sentence[-i]
        # Encode and store word
        seed_encoded[0][-i] = ds.encoder.encode(word_decoded)
    # Parse encoded seed to tensor
    seed_encoded = torch.tensor(seed_encoded).float()
    
    # Indicate output start
    print(seed_text, end=' ', flush=True)
    # Initialize network input, output and state
    net_in, net_out, net_state = seed_encoded, None, None
    # Continue looping until all requested words have been returned
    for i in range(words_len):
        # Get current network output and state
        net_out, net_state = net(net_in, net_state)
        # Get current encoded word
        word_encoded = net_out[0, -1, :].numpy()
        # Decude current encoded word
        word_decoded = ds.encoder.decode(word_encoded)
        # Get current word (decode output)
        print(word_decoded, end=' ', flush=True)
        # Update next network input
        net_in = net_out

i don't know None
spouted (tensor([[[ 1.7513e-07,  1.8925e-07, -6.2418e-08, -1.5110e-05, -9.9738e-07,
           2.7302e-04, -2.6995e-05,  1.3441e-07, -6.9002e-05,  1.1651e-09,
          -5.6402e-07, -1.0355e-04, -2.3787e-07,  5.1738e-04, -1.4997e-07,
           7.7710e-08, -4.6949e-09,  5.4700e-07,  6.7430e-08, -6.5716e-08,
          -5.6395e-09, -1.9366e-04,  2.4158e-06, -4.3430e-08, -3.5821e-04,
          -6.9595e-07,  2.4696e-05,  3.3230e-05,  3.9764e-05,  2.4486e-07,
          -5.2795e-07, -6.4229e-07,  4.7816e-07,  6.8324e-07, -2.1714e-06,
           2.6247e-07,  1.0232e-03, -1.7298e-05, -2.2422e-07,  1.5632e-07,
          -4.4457e-07,  6.4597e-06, -2.8375e-05,  5.1466e-05,  2.1980e-04,
           9.5366e-07,  1.2527e-07, -5.3579e-04, -1.1758e-07, -2.3013e-07,
          -2.2826e-07,  4.2177e-04, -5.6952e-05, -8.0822e-06,  1.3162e-04,
          -3.4414e-07, -7.8166e-07, -2.7868e-06,  1.0314e-07,  3.7932e-07,
           1.9472e-07,  4.7729e-07, -2.5936e-08,  2.4019e-04,  1.4294e-07

spouted (tensor([[[ 5.8581e-08,  2.1664e-08, -9.4613e-08, -1.6795e-05, -1.8258e-06,
           2.9020e-04, -2.8593e-05,  6.2293e-08, -7.2894e-05,  2.1245e-08,
          -6.1728e-07, -1.0980e-04, -3.0619e-07,  5.5001e-04,  8.8294e-08,
           7.6866e-08, -1.1127e-07,  5.0572e-07,  8.4383e-08,  2.6111e-07,
           2.8501e-08, -2.0680e-04,  2.6625e-06, -3.0216e-07, -3.8079e-04,
          -4.2276e-07,  2.6258e-05,  3.5412e-05,  4.3208e-05,  4.4533e-08,
          -3.8440e-07, -5.8551e-07,  4.0732e-07,  7.4073e-07, -2.5575e-06,
           1.2432e-07,  1.0873e-03, -1.9061e-05, -1.7481e-07,  7.2873e-07,
          -6.2187e-07,  7.1849e-06, -3.0096e-05,  5.4623e-05,  2.3281e-04,
           8.6695e-07,  1.0579e-07, -5.6924e-04,  6.9920e-07, -1.1947e-07,
          -1.6150e-07,  4.4813e-04, -6.0267e-05, -8.2867e-06,  1.3979e-04,
          -1.4257e-07, -8.6484e-07, -2.7241e-06,  2.3074e-07,  4.8607e-07,
          -4.5527e-08,  6.4740e-07, -2.7931e-08,  2.5527e-04,  5.5150e-07,
           2.361

spouted (tensor([[[ 5.1904e-08,  1.1652e-08, -9.6897e-08, -1.6913e-05, -1.9054e-06,
           2.9128e-04, -2.8692e-05,  5.8209e-08, -7.3138e-05,  2.3225e-08,
          -6.3493e-07, -1.1019e-04, -3.1180e-07,  5.5204e-04,  1.0410e-07,
           7.7833e-08, -1.1932e-07,  5.0292e-07,  8.5495e-08,  2.8328e-07,
           3.1406e-08, -2.0763e-04,  2.6712e-06, -3.1685e-07, -3.8220e-04,
          -4.0571e-07,  2.6356e-05,  3.5549e-05,  4.3422e-05,  3.1702e-08,
          -3.7518e-07, -5.9007e-07,  4.0422e-07,  7.4666e-07, -2.5817e-06,
           1.1810e-07,  1.0913e-03, -1.9173e-05, -1.7119e-07,  7.6449e-07,
          -6.4297e-07,  7.2329e-06, -3.0201e-05,  5.4821e-05,  2.3362e-04,
           8.5870e-07,  1.0494e-07, -5.7133e-04,  7.5011e-07, -1.1419e-07,
          -1.6210e-07,  4.4978e-04, -6.0475e-05, -8.2995e-06,  1.4030e-04,
          -1.3104e-07, -8.7017e-07, -2.7231e-06,  2.4088e-07,  4.9971e-07,
          -6.0005e-08,  6.6769e-07, -2.8361e-08,  2.5622e-04,  5.7800e-07,
           3.401

spouted (tensor([[[ 5.1526e-08,  1.1057e-08, -9.7057e-08, -1.6921e-05, -1.9121e-06,
           2.9135e-04, -2.8698e-05,  5.7981e-08, -7.3153e-05,  2.3394e-08,
          -6.3693e-07, -1.1021e-04, -3.1223e-07,  5.5217e-04,  1.0515e-07,
           7.7955e-08, -1.1992e-07,  5.0273e-07,  8.5569e-08,  2.8478e-07,
           3.1632e-08, -2.0768e-04,  2.6713e-06, -3.1768e-07, -3.8229e-04,
          -4.0464e-07,  2.6362e-05,  3.5557e-05,  4.3435e-05,  3.0879e-08,
          -3.7459e-07, -5.9086e-07,  4.0411e-07,  7.4718e-07, -2.5832e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7093e-07,  7.6672e-07,
          -6.4491e-07,  7.2361e-06, -3.0207e-05,  5.4833e-05,  2.3367e-04,
           8.5801e-07,  1.0490e-07, -5.7146e-04,  7.5328e-07, -1.1396e-07,
          -1.6244e-07,  4.4989e-04, -6.0488e-05, -8.3003e-06,  1.4033e-04,
          -1.3038e-07, -8.7051e-07, -2.7232e-06,  2.4164e-07,  5.0099e-07,
          -6.0876e-08,  6.6956e-07, -2.8408e-08,  2.5628e-04,  5.7972e-07,
           3.457

spouted (tensor([[[ 5.1506e-08,  1.1022e-08, -9.7068e-08, -1.6921e-05, -1.9126e-06,
           2.9135e-04, -2.8698e-05,  5.7968e-08, -7.3154e-05,  2.3408e-08,
          -6.3711e-07, -1.1022e-04, -3.1226e-07,  5.5218e-04,  1.0521e-07,
           7.7967e-08, -1.1996e-07,  5.0272e-07,  8.5574e-08,  2.8488e-07,
           3.1648e-08, -2.0768e-04,  2.6713e-06, -3.1772e-07, -3.8229e-04,
          -4.0457e-07,  2.6362e-05,  3.5558e-05,  4.3436e-05,  3.0826e-08,
          -3.7456e-07, -5.9094e-07,  4.0411e-07,  7.4722e-07, -2.5833e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7091e-07,  7.6686e-07,
          -6.4507e-07,  7.2363e-06, -3.0207e-05,  5.4834e-05,  2.3368e-04,
           8.5795e-07,  1.0490e-07, -5.7147e-04,  7.5348e-07, -1.1395e-07,
          -1.6248e-07,  4.4989e-04, -6.0488e-05, -8.3003e-06,  1.4033e-04,
          -1.3035e-07, -8.7053e-07, -2.7232e-06,  2.4170e-07,  5.0110e-07,
          -6.0928e-08,  6.6971e-07, -2.8412e-08,  2.5628e-04,  5.7983e-07,
           3.460

...

spouted (tensor([[[ 5.1504e-08,  1.1020e-08, -9.7069e-08, -1.6921e-05, -1.9127e-06,
           2.9135e-04, -2.8698e-05,  5.7968e-08, -7.3154e-05,  2.3409e-08,
          -6.3713e-07, -1.1022e-04, -3.1227e-07,  5.5218e-04,  1.0522e-07,
           7.7968e-08, -1.1996e-07,  5.0272e-07,  8.5575e-08,  2.8489e-07,
           3.1649e-08, -2.0768e-04,  2.6713e-06, -3.1772e-07, -3.8230e-04,
          -4.0457e-07,  2.6362e-05,  3.5558e-05,  4.3436e-05,  3.0822e-08,
          -3.7455e-07, -5.9095e-07,  4.0411e-07,  7.4722e-07, -2.5833e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7091e-07,  7.6687e-07,
          -6.4509e-07,  7.2363e-06, -3.0208e-05,  5.4834e-05,  2.3368e-04,
           8.5795e-07,  1.0490e-07, -5.7147e-04,  7.5349e-07, -1.1395e-07,
          -1.6249e-07,  4.4989e-04, -6.0489e-05, -8.3003e-06,  1.4033e-04,
          -1.3034e-07, -8.7053e-07, -2.7232e-06,  2.4170e-07,  5.0111e-07,
          -6.0931e-08,  6.6973e-07, -2.8413e-08,  2.5628e-04,  5.7984e-07,
           3.460

spouted (tensor([[[ 5.1504e-08,  1.1020e-08, -9.7069e-08, -1.6921e-05, -1.9127e-06,
           2.9135e-04, -2.8698e-05,  5.7968e-08, -7.3154e-05,  2.3409e-08,
          -6.3713e-07, -1.1022e-04, -3.1227e-07,  5.5218e-04,  1.0522e-07,
           7.7968e-08, -1.1996e-07,  5.0272e-07,  8.5575e-08,  2.8489e-07,
           3.1649e-08, -2.0768e-04,  2.6713e-06, -3.1772e-07, -3.8230e-04,
          -4.0457e-07,  2.6362e-05,  3.5558e-05,  4.3436e-05,  3.0822e-08,
          -3.7455e-07, -5.9095e-07,  4.0411e-07,  7.4722e-07, -2.5833e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7091e-07,  7.6687e-07,
          -6.4509e-07,  7.2363e-06, -3.0208e-05,  5.4834e-05,  2.3368e-04,
           8.5795e-07,  1.0490e-07, -5.7147e-04,  7.5349e-07, -1.1395e-07,
          -1.6249e-07,  4.4989e-04, -6.0489e-05, -8.3003e-06,  1.4033e-04,
          -1.3034e-07, -8.7053e-07, -2.7232e-06,  2.4170e-07,  5.0111e-07,
          -6.0931e-08,  6.6973e-07, -2.8413e-08,  2.5628e-04,  5.7984e-07,
           3.460

spouted (tensor([[[ 5.1504e-08,  1.1020e-08, -9.7069e-08, -1.6921e-05, -1.9127e-06,
           2.9135e-04, -2.8698e-05,  5.7968e-08, -7.3154e-05,  2.3409e-08,
          -6.3713e-07, -1.1022e-04, -3.1227e-07,  5.5218e-04,  1.0522e-07,
           7.7968e-08, -1.1996e-07,  5.0272e-07,  8.5575e-08,  2.8489e-07,
           3.1649e-08, -2.0768e-04,  2.6713e-06, -3.1772e-07, -3.8230e-04,
          -4.0457e-07,  2.6362e-05,  3.5558e-05,  4.3436e-05,  3.0822e-08,
          -3.7455e-07, -5.9095e-07,  4.0411e-07,  7.4722e-07, -2.5833e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7091e-07,  7.6687e-07,
          -6.4509e-07,  7.2363e-06, -3.0208e-05,  5.4834e-05,  2.3368e-04,
           8.5795e-07,  1.0490e-07, -5.7147e-04,  7.5349e-07, -1.1395e-07,
          -1.6249e-07,  4.4989e-04, -6.0489e-05, -8.3003e-06,  1.4033e-04,
          -1.3034e-07, -8.7053e-07, -2.7232e-06,  2.4170e-07,  5.0111e-07,
          -6.0931e-08,  6.6973e-07, -2.8413e-08,  2.5628e-04,  5.7984e-07,
           3.460

spouted (tensor([[[ 5.1504e-08,  1.1020e-08, -9.7069e-08, -1.6921e-05, -1.9127e-06,
           2.9135e-04, -2.8698e-05,  5.7968e-08, -7.3154e-05,  2.3409e-08,
          -6.3713e-07, -1.1022e-04, -3.1227e-07,  5.5218e-04,  1.0522e-07,
           7.7968e-08, -1.1996e-07,  5.0272e-07,  8.5575e-08,  2.8489e-07,
           3.1649e-08, -2.0768e-04,  2.6713e-06, -3.1772e-07, -3.8230e-04,
          -4.0457e-07,  2.6362e-05,  3.5558e-05,  4.3436e-05,  3.0822e-08,
          -3.7455e-07, -5.9095e-07,  4.0411e-07,  7.4722e-07, -2.5833e-06,
           1.1786e-07,  1.0915e-03, -1.9180e-05, -1.7091e-07,  7.6687e-07,
          -6.4509e-07,  7.2363e-06, -3.0208e-05,  5.4834e-05,  2.3368e-04,
           8.5795e-07,  1.0490e-07, -5.7147e-04,  7.5349e-07, -1.1395e-07,
          -1.6249e-07,  4.4989e-04, -6.0489e-05, -8.3003e-06,  1.4033e-04,
          -1.3034e-07, -8.7053e-07, -2.7232e-06,  2.4170e-07,  5.0111e-07,
          -6.0931e-08,  6.6973e-07, -2.8413e-08,  2.5628e-04,  5.7984e-07,
           3.460

spouted 